In [44]:
import numpy as np
import pandas as pd
import re
import requests #Scrapping
from bs4 import BeautifulSoup # Scrapping
import time #For Time Stamps
import nltk
nltk.download('punkt')
import matplotlib.pyplot as plt
from datetime import datetime
!pip install textblob # Needed for Sentiment Code
!pip install newspaper3k # Needed for Sentiment Code
# The punkt download enabled nlp() as a function to be done. 
from textblob import TextBlob # Sentiment Code
from newspaper import Article #Sentiment Code
!pip3 install --upgrade tensorflow-gpu --user
!pip install yfinance
!pip install yahoofinancials
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
!pip install newsapi-python
from newsapi import NewsApiClient
#from key import my_api_key
import datetime as dt
newsapi = NewsApiClient(api_key='f7ac29c049dc4e5f96aa077cf36a612d')



from io import StringIO
from datetime import datetime, timedelta

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DPoll\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
ERROR: Invalid requirement: '#'
ERROR: Invalid requirement: '#'


  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
    Running setup.py install for tensorflow-gpu: started
    Running setup.py install for tensorflow-gpu: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\DPoll\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\DPoll\\AppData\\Local\\Temp\\pip-install-yq0xheo2\\tensorflow-gpu_746b66541b354886be0d1729cadc1c3a\\setup.py'"'"'; __file__='"'"'C:\\Users\\DPoll\\AppData\\Local\\Temp\\pip-install-yq0xheo2\\tensorflow-gpu_746b66541b354886be0d1729cadc1c3a\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\DPoll\AppData\Local\Temp\pip-wheel-qa1liv8h'
       cwd: C:\Users\DPoll\AppData\Local\Temp\pip-install-yq0xheo2\tensorflow-gpu_746b66541b354886be0d1729cadc1c3a\
  Complete output (17 lines):
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "C:\Us

In [33]:
data=[]

search_strings = ["MO"]

links=[]

for srch in search_strings:
    searchlinks = "https://finance.yahoo.com/quote/" + srch + "/history?p=" + srch
    links.append(searchlinks)
    print(searchlinks)
print(links)


for currentpage in links:
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:75.0) Gecko/20100101 Firefox/75.0'}
    print("**Now Scraping:",currentpage)
    currentpage_resp=requests.get(currentpage, headers=headers)
    #html = currentpage_resp.content
    currentpagesoup=BeautifulSoup(currentpage_resp.content,"html.parser") 
    stockdata=currentpagesoup.find_all('span') 
    for stock in stockdata:
        stock = stock.find('a', href = True)
        if stock != None:
            print(stock)

https://finance.yahoo.com/quote/MO/history?p=MO
['https://finance.yahoo.com/quote/MO/history?p=MO']
**Now Scraping: https://finance.yahoo.com/quote/MO/history?p=MO
<a class="Fl(end) Mt(3px) Cur(p)" download="MO.csv" href="https://query1.finance.yahoo.com/v7/finance/download/MO?period1=1649623941&amp;period2=1681159941&amp;interval=1d&amp;events=history&amp;includeAdjustedClose=true"><svg class="Va(m)! Mend(5px) Stk($linkColor)! Fill($linkColor)! Cur(p)" data-icon="download" height="15" style="fill:#0081f2;stroke:#0081f2;stroke-width:0;vertical-align:bottom" viewbox="0 0 24 24" width="15"><path d="M21 20c.552 0 1 .448 1 1s-.448 1-1 1H3c-.552 0-1-.448-1-1s.448-1 1-1h18zM13 2v12.64l3.358-3.356c.375-.375.982-.375 1.357 0s.375.983 0 1.357L12 18l-5.715-5.36c-.375-.373-.375-.98 0-1.356.375-.375.983-.375 1.358 0L11 14.64V2h2z"></path></svg><span>Download</span></a>


In [34]:
for currentpage in links:
    print("**Now Scraping:",currentpage)
    
    currentpage_resp=requests.get(currentpage)
    
    currentpagesoup=BeautifulSoup(currentpage_resp.content,"html.parser")
    
    articles=currentpagesoup.find_all('div',class_="search-result-indiv") 

    if articles != []:
        for article in articles:
            try:
                heading=article.find("h3", class_="search-result-title").get_text().strip()
            except:
                heading="NA"
            print(heading)
            try:
                dt=article.find("h5", class_="search-result-timestamp").get_text().strip()
            except:
                dt="NA"
            print(dt)
            try:
                hyperlink=article.find("a").attrs['href']
                hyperlink2= "https://www.reuters.com"+hyperlink
            except:
                hyperlink2="NA"
            print(hyperlink2)
            
            
            data.append((heading,dt,hyperlink2))
    else:
        print("------The search yieled no results-----")
    
if data!=[]:
    df=pd.DataFrame(np.array(data))
    df.columns=["headline","dateandtime","link"]
    print(df)
    df.to_csv('question9.csv')

**Now Scraping: https://finance.yahoo.com/quote/MO/history?p=MO
------The search yieled no results-----


In [35]:
class YahooFinanceHistory:
    timeout = 2
    crumb_link = 'https://finance.yahoo.com/quote/{0}/history?p={0}'
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    quote_link = 'https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1d&events=history&crumb={crumb}'

    def __init__(self, symbol, days_back=7):
        self.symbol = symbol
        self.session = requests.Session()
        self.dt = timedelta(days=days_back)

    def get_crumb(self):
        response = self.session.get(self.crumb_link.format(self.symbol), timeout=self.timeout)
        response.raise_for_status()
        match = re.search(self.crumble_regex, response.text)
        if not match:
            raise ValueError('Could not get crumb from Yahoo Finance')
        else:
            self.crumb = match.group(1)

    def get_quote(self):
        if not hasattr(self, 'crumb') or len(self.session.cookies) == 0:
            self.get_crumb()
        now = datetime.utcnow()
        dateto = int(now.timestamp())
        datefrom = int((now - self.dt).timestamp())
        url = self.quote_link.format(quote=self.symbol, dfrom=datefrom, dto=dateto, crumb=self.crumb)
        response = self.session.get(url)
        response.raise_for_status()
        return pd.read_csv(StringIO(response.text), parse_dates=['Date'])

In [36]:
df1 = yf.download('AAPL')
print(df1)

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
1980-12-12    0.128348    0.128906    0.128348    0.128348    0.099722   
1980-12-15    0.122210    0.122210    0.121652    0.121652    0.094519   
1980-12-16    0.113281    0.113281    0.112723    0.112723    0.087582   
1980-12-17    0.115513    0.116071    0.115513    0.115513    0.089749   
1980-12-18    0.118862    0.119420    0.118862    0.118862    0.092351   
...                ...         ...         ...         ...         ...   
2023-04-03  164.270004  166.289993  164.220001  166.169998  166.169998   
2023-04-04  166.600006  166.839996  165.110001  165.630005  165.630005   
2023-04-05  164.740005  165.050003  161.800003  163.759995  163.759995   
2023-04-06  162.429993  164.960007  162.000000  164.660004  164.660004   
2023-04-10  162.429993  162.029999  160.080

In [37]:
# /v2/top-headlines
top_headlines = newsapi.get_top_headlines(q='MO',
                                          sources='bbc-news,the-verge',
                                          language='en')

# /v2/everything
all_articles = newsapi.get_everything(q='MO',
                                      sources='bbc-news,the-verge',
                                      domains='bbc.co.uk,techcrunch.com',
                                      from_param='2023-03-11',
                                      to='2023-04-10',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)
print(all_articles)
# /v2/top-headlines/sources
sources = newsapi.get_sources()

{'status': 'ok', 'totalResults': 20, 'articles': []}


In [38]:
# /v2/everything
data = newsapi.get_everything(q='Altria Group Inc', language='en', page_size = 20)
print(data)
print(type(data))
print(data.keys())
print(data['totalResults'])

{'status': 'ok', 'totalResults': 62, 'articles': [{'source': {'id': None, 'name': 'Yahoo Entertainment'}, 'author': 'Soumya Eswaran', 'title': 'Should You Sell Altria Group (MO) Now?', 'description': 'Broyhill Asset Management, a boutique investment firm, released its fourth-quarter 2022 investor letter. A copy of the same can be downloaded here. The...', 'url': 'https://finance.yahoo.com/news/sell-altria-group-mo-now-101420404.html', 'urlToImage': 'https://media.zenfs.com/en/insidermonkey.com/a1e348cc6d3ea31dd5ab2e6e7514aaa3', 'publishedAt': '2023-03-20T10:14:20Z', 'content': 'Broyhill Asset Management, a boutique investment firm, released its fourth-quarter 2022 investor letter. A copy of the same can be downloaded here. The portfolio gained 1.3% net of fees and expenses … [+1875 chars]'}, {'source': {'id': None, 'name': 'Yahoo Entertainment'}, 'author': 'Simply Wall St', 'title': 'Do These 3 Checks Before Buying Altria Group, Inc. (NYSE:MO) For Its Upcoming Dividend', 'description':

In [39]:
print(data['articles'][0])

{'source': {'id': None, 'name': 'Yahoo Entertainment'}, 'author': 'Soumya Eswaran', 'title': 'Should You Sell Altria Group (MO) Now?', 'description': 'Broyhill Asset Management, a boutique investment firm, released its fourth-quarter 2022 investor letter. A copy of the same can be downloaded here. The...', 'url': 'https://finance.yahoo.com/news/sell-altria-group-mo-now-101420404.html', 'urlToImage': 'https://media.zenfs.com/en/insidermonkey.com/a1e348cc6d3ea31dd5ab2e6e7514aaa3', 'publishedAt': '2023-03-20T10:14:20Z', 'content': 'Broyhill Asset Management, a boutique investment firm, released its fourth-quarter 2022 investor letter. A copy of the same can be downloaded here. The portfolio gained 1.3% net of fees and expenses … [+1875 chars]'}


In [40]:
articles = data['articles']

In [41]:
for x, y in enumerate(articles):
    print(f'{x}    {y["title"]}')

0    Should You Sell Altria Group (MO) Now?
1    Do These 3 Checks Before Buying Altria Group, Inc. (NYSE:MO) For Its Upcoming Dividend
2    Juul Is Divested - Altria Now Focuses On Development Efforts For Smoke-Free Products, Updates 2028 Goals
3    Juul, Altria Face First Trial Over Claims Of Marketing E-cigarettes To Teens
4    Juul reaches settlement with Chicago for nearly $24 million
5    Kaival Brands Appoints Three New Members to its Board of Directors
6    Juul, Altria Face First Trial Over Claims of Marketing E-Cigarettes to Teens
7    Seeking at Least 8% Dividend Yield? Goldman Sachs Suggests 2 Dividend Stocks to Buy
8    25 Best Stocks For Dividends
9    Monday’s Top Analyst Upgrades and Downgrades: Altria, Block, Coinbase Global, Deere, First Horizon, KB Home, Pfizer, PNC Financial Services, RingCentral, Uber Technologies, Zoom Video Communications
10    You'll Never Guess the Top-Performing Stock of the Last 20 Years
11    Economic Investment Alliance Launches, Urges Full

In [42]:
for key, value in articles[0].items():
    print(f'\n{key.ljust(15)} {value}')


source          {'id': None, 'name': 'Yahoo Entertainment'}

author          Soumya Eswaran

title           Should You Sell Altria Group (MO) Now?

description     Broyhill Asset Management, a boutique investment firm, released its fourth-quarter 2022 investor letter. A copy of the same can be downloaded here. The...

url             https://finance.yahoo.com/news/sell-altria-group-mo-now-101420404.html

urlToImage      https://media.zenfs.com/en/insidermonkey.com/a1e348cc6d3ea31dd5ab2e6e7514aaa3

publishedAt     2023-03-20T10:14:20Z

content         Broyhill Asset Management, a boutique investment firm, released its fourth-quarter 2022 investor letter. A copy of the same can be downloaded here. The portfolio gained 1.3% net of fees and expenses … [+1875 chars]


In [45]:
pub_date = dt.datetime.strptime(articles[0]['publishedAt'], '%Y-%m-%dT%H:%M:%SZ').date()
print(pub_date)
print(pub_date.year)
print(pub_date.month)
print(pub_date.day)

2023-03-20
2023
3
20


In [47]:
df = pd.DataFrame(articles)
print(df)

                                               source  \
0         {'id': None, 'name': 'Yahoo Entertainment'}   
1         {'id': None, 'name': 'Yahoo Entertainment'}   
2         {'id': None, 'name': 'Yahoo Entertainment'}   
3   {'id': None, 'name': 'International Business T...   
4         {'id': None, 'name': 'Yahoo Entertainment'}   
5         {'id': None, 'name': 'Yahoo Entertainment'}   
6           {'id': None, 'name': 'Insurance Journal'}   
7         {'id': None, 'name': 'Yahoo Entertainment'}   
8         {'id': None, 'name': 'Yahoo Entertainment'}   
9               {'id': None, 'name': '24/7 Wall St.'}   
10        {'id': None, 'name': 'Yahoo Entertainment'}   
11        {'id': None, 'name': 'Yahoo Entertainment'}   
12              {'id': None, 'name': '24/7 Wall St.'}   
13              {'id': None, 'name': '24/7 Wall St.'}   
14              {'id': None, 'name': '24/7 Wall St.'}   
15        {'id': None, 'name': 'Yahoo Entertainment'}   
16        {'id': None, 'name': 